In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))
postulaciones = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulaciones/Postulaciones-merge.csv',
                            dtype=column_types,parse_dates=['fechapostulacion'],infer_datetime_format=True)
postulaciones = postulaciones.drop(columns=['Unnamed: 0','fechapostulacion'])

In [3]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 2 columns):
idaviso         uint32
idpostulante    object
dtypes: object(1), uint32(1)
memory usage: 75.6+ MB


In [4]:
dtypes = pd.Series({'idpostulante': np.dtype('object'), 'sexo': CategoricalDtype(categories=['FEM', 'MASC', 'NO_DECLARA'], ordered=False)})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_genero_edad = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulantes-Genero-Edad/postulantes_genero_y_edad-merge.csv',
                                    dtype=column_types,parse_dates=['fechanacimiento'],
                                      infer_datetime_format=True)
#postulantes_genero_edad = postulantes_genero_edad.drop(columns=['Unnamed: 0'],axis=1)
#postulantes_genero_edad

In [6]:
dtypes = pd.Series({'estado': CategoricalDtype(categories=['Abandonado', 'En Curso', 'Graduado'], ordered=False), 'idpostulante': np.dtype('object'),
                    'nombre': CategoricalDtype(categories=['Doctorado', 'Master', 'Otro', 'Posgrado', 'Secundario',
                  'Terciario/Técnico', 'Universitario'],
                 ordered=False)})


dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

postulantes_educacion = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/Postulantes-Educacion/postulantes_educacion-merge.csv',
                                    dtype=column_types)
postulantes_educacion = postulantes_educacion.drop(columns=['nombre_numerico'])

In [7]:
#Hago un merge de educacion y genero-edad
postulantes_merge = pd.merge(postulantes_genero_edad,postulantes_educacion,how='left',on='idpostulante')

In [8]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504400 entries, 0 to 504399
Data columns (total 6 columns):
idpostulante       504400 non-null object
fechanacimiento    504400 non-null object
sexo               504400 non-null category
sexo_numerico      504400 non-null int64
nombre             447906 non-null category
estado             447906 non-null category
dtypes: category(3), int64(1), object(2)
memory usage: 16.8+ MB


In [9]:
postulantes_merge.head()

,idpostulante,fechanacimiento,sexo,sexo_numerico,nombre,estado
0,zvxzzL6,1989-08-31,MASC,2,Secundario,Graduado
1,zvxzvD5,1994-10-10,FEM,1,Terciario/Técnico,En Curso
2,zvxzrWm,1981-11-30,MASC,2,Universitario,Graduado
3,zvxzmED,1999-04-09,FEM,1,Secundario,Graduado
4,zvxzld5,1997-12-17,MASC,2,NaN,NaN


In [10]:
#Voy a agregar una columna con la edad ya calculada(estimando por 2017-año nacimiento) para cada registro
postulantes_merge["año"] = postulantes_merge[postulantes_merge["fechanacimiento"] != 'nan']\
        ["fechanacimiento"].apply(lambda x: x[0:4])
postulantes_merge["edad"] = postulantes_merge[postulantes_merge["año"].notna()] \
        ["año"].apply(lambda x: 2017 - int(x))

In [11]:
#Elimino columna fechanacimiento y año que ya no me sirven
postulantes_merge = postulantes_merge.drop(columns=['fechanacimiento','año'])

In [12]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504400 entries, 0 to 504399
Data columns (total 6 columns):
idpostulante     504400 non-null object
sexo             504400 non-null category
sexo_numerico    504400 non-null int64
nombre           447906 non-null category
estado           447906 non-null category
edad             478315 non-null float64
dtypes: category(3), float64(1), int64(1), object(1)
memory usage: 16.8+ MB


In [13]:
#Elimino NaNs
postulantes_merge = postulantes_merge[postulantes_merge['nombre'].notnull() &
                                     postulantes_merge['estado'].notnull() &
                                     postulantes_merge['edad'].notnull()]

In [14]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438702 entries, 0 to 504399
Data columns (total 6 columns):
idpostulante     438702 non-null object
sexo             438702 non-null category
sexo_numerico    438702 non-null int64
nombre           438702 non-null category
estado           438702 non-null category
edad             438702 non-null float64
dtypes: category(3), float64(1), int64(1), object(1)
memory usage: 14.6+ MB


In [15]:
#Vamos a codificar cada columna que sea de strings
lb_make1 = LabelEncoder()
postulantes_merge["estado_code"] = lb_make1.fit_transform(postulantes_merge["estado"])

lb_make2 = LabelEncoder()
postulantes_merge["sexo_code"] = lb_make2.fit_transform(postulantes_merge["sexo"])

lb_make3 = LabelEncoder()
postulantes_merge["nombre_code"] = lb_make3.fit_transform(postulantes_merge["nombre"])

postulantes_merge.drop(columns=['sexo','nombre','estado'],inplace=True)
postulantes_merge[['sexo_code','nombre_code','estado_code','edad']] = postulantes_merge[['sexo_code','nombre_code','estado_code','edad']].apply(pd.to_numeric,downcast='unsigned')

In [16]:
postulantes_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438702 entries, 0 to 504399
Data columns (total 6 columns):
idpostulante     438702 non-null object
sexo_numerico    438702 non-null int64
edad             438702 non-null uint16
estado_code      438702 non-null uint8
sexo_code        438702 non-null uint8
nombre_code      438702 non-null uint8
dtypes: int64(1), object(1), uint16(1), uint8(3)
memory usage: 12.1+ MB


In [17]:
postulantes_merge.to_csv('/home/fabrizio/env/NaventDatosTP/Data/POSTULANTES_FINAL.csv')